# 모델 매개변수 최적화하기
* 데이터에 대해 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트 할 차례
* 각 반복 단계에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류(손실)를 계산하고,
* 매개변수에 대한 오류의 도함수를 수집한 뒤, 경사하강법을 사용하여 이 파라미터들을 optimize한다.

In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [17]:
class myNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(28*28, 512)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        return x
        
model = myNet()

In [18]:
from torchsummary import summary

summary(model, (28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                   [-1, 10]           5,130
              ReLU-5                   [-1, 10]               0
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 1.55
Estimated Total Size (MB): 1.57
----------------------------------------------------------------


In [19]:
size = len(train_dataloader.dataset)
print(size)

60000


In [20]:
train_dataloader.dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [21]:
feature, label = next(iter(train_dataloader))

In [22]:
print(feature.shape)

torch.Size([64, 1, 28, 28])


# Hyperparameter
* learning rate
* epoch
* batch_size
* loss function
* optimizer
* 위 4가지 개념에 대한 설명은 생략
* optimizer는 코드가 좀 중요해 보여서 적어두겠음

In [23]:
learning_rate = 1e-3
epochs = 5
batch_size = 64

In [24]:
# 손실 함수
loss_fn = nn.CrossEntropyLoss()

## Optimizer
* 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정이다.
* 최적화 알고리즘은 이 과정이 수행되는 방식(여기에서는 SGD)를 정의 한다.
* <b>모든 최적화 절차는 optimizer 객체에 캡슐화된다.</b>
* 여기서는 SGD 옵티마이저를 사용하고 있으며, PyTorch에는 ADAM이나 RMSProp과 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있다.

* 학습하려는 모델의 매개변수와 learning rate 하이퍼 파라미터를 등록하여 옵티마이저를 초기화 한다.

In [25]:
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

* 학습 단계(loop)에서 최적화는 세단계로 이뤄집니다.
    * <code>optimizer.zero_grad()</code>를 호출하여 모델 매개변수의 변화도를 재설정한다. 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정한다.
        * 각 Tensor에 대해 이미 .grad라는 속성에 대해서 optimizer가 연산을 하는 것인가?
    * <code>loss.backward()</code>를 호출하여 prediction loss를 역전파한다.
    * 변화도를 계산한 뒤에는 <code>optimzer.step()</code>을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정한다.

### 번외) dataLoader에서 data 뽑아내는 법
* <code>iter(), next()</code>를 통해서 뽑아낸다.
* 이는 iterable한 object에 대해서 <code>object.\_\_iter\_\_()</code>과 <code>object.\_\_next\_\_()</code>를 호출하는 것과 같다.

In [29]:
data = train_dataloader.__iter__()
# == data = iter(train_dataloader)
print(data.__next__())
# == print(next(data))

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0

In [31]:
print(train_dataloader.dataset.__len__())

60000


In [40]:
import numpy as np

li = np.array([3, 5, 6, 8])
print(type(li.argmax()))

<class 'numpy.int64'>


In [48]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    # 1 Epoch
    for batch, (X, y) in enumerate(dataloader):
        # Forward Propagtion
        # 예측(Prediction)과 loss 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Back Propagation
        optimizer.zero_grad() # 변화 값 초기화
        loss.backward() # loss에서 back propagation
        optimizer.step() # gradient에 대한 optimzer의 계산
        
        if(batch % 100 == 0):
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # 데이터셋 크기
    num_batches = len(dataloader) # Batch Size
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 여기 문법이 어떻게 이렇게 되지
            # print((pred.argmax(1) == y))
            # print((pred.argmax(1) == y).type(torch.float))
            # print((pred.argmax(1) == y).type(torch.float).sum()) # Batch라서 Sum이 필요했음
            # print((pred.argmax(1) == y).type(torch.float).sum().item()) # Sum(Aggregate)를 했으니 원소 값을 가져오는 item() 쓴 거
            
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [49]:
loss_fn = nn.CrossEntropyLoss()
optmizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10

for t in range(epochs):
    print(f'Epoch {t + 1}\n---------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
---------------------------
loss: 1.554697 [   64/60000]
loss: 1.447941 [ 6464/60000]
loss: 1.420570 [12864/60000]
loss: 1.601423 [19264/60000]
loss: 1.384752 [25664/60000]
loss: 1.480931 [32064/60000]
loss: 1.538543 [38464/60000]
loss: 1.556124 [44864/60000]
loss: 1.491038 [51264/60000]
loss: 1.415847 [57664/60000]
Test Error: 
 Accuracy: 54.5%, Avg loss: 1.450528 

Epoch 2
---------------------------
loss: 1.433963 [   64/60000]
loss: 1.333974 [ 6464/60000]
loss: 1.302448 [12864/60000]
loss: 1.511111 [19264/60000]
loss: 1.281652 [25664/60000]
loss: 1.396314 [32064/60000]
loss: 1.458098 [38464/60000]
loss: 1.489570 [44864/60000]
loss: 1.408708 [51264/60000]
loss: 1.348030 [57664/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.372590 

Epoch 3
---------------------------
loss: 1.348174 [   64/60000]
loss: 1.255614 [ 6464/60000]
loss: 1.218817 [12864/60000]
loss: 1.447169 [19264/60000]
loss: 1.210530 [25664/60000]
loss: 1.336095 [32064/60000]
loss: 1.402094 [38464/60000]
loss: